In [1]:
import numpy
print(numpy.__version__)


1.26.1


In [3]:
pip install --upgrade numpy

  Using cached numpy-1.26.1-cp39-cp39-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 1.26.1 which is incompatible.
pandas-profiling 3.1.0 requires joblib~=1.0.1, but you have joblib 1.2.0 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.1 which is incompatible.


In [4]:
pip install --upgrade opencv-python

  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl (38.1 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\HP\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [5]:
pip install opencv-python

  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl (38.1 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\HP\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
import cv2

In [2]:
import os

In [3]:
data_path='Dataset'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

{'female': 0, 'male': 1}
['female', 'male']
[0, 1]


In [4]:
data_path

'Dataset'

In [5]:
img_size=32
data=[]
target=[]

facedata = "haarcascade_frontalface_default (1).xml"
cascade = cv2.CascadeClassifier(facedata)


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        faces = cascade.detectMultiScale(img)
        try:
            for f in faces:
                x, y, w, h = [v for v in f]
                sub_face = img[y:y + h, x:x + w]
                gray=cv2.cvtColor(sub_face,cv2.COLOR_BGR2GRAY)           
                resized=cv2.resize(gray,(img_size,img_size))
                data.append(resized)
                target.append(label_dict[category])
        except Exception as e:
            print('Exception:',e)

In [10]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [7]:
import tensorflow as tf
from tensorflow import keras

In [8]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data = np.array(data)/255.0
data = np.reshape(data, (data.shape[0], img_size, img_size, 1))
target = np.array(target)

In [9]:
from tensorflow.keras.utils import to_categorical

new_target = to_categorical(target)

np.save('./training/data',data)
np.save('./training/target',new_target)

In [10]:
import numpy as np

data=np.load('./training/data.npy')
target=np.load('./training/target.npy')

In [11]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

noOfFilters=64
sizeOfFilter1=(3,3)
sizeOfFilter2=(3,3)
sizeOfPool=(2,2)
noOfNode=64



model=Sequential()
model.add((Conv2D(32, sizeOfFilter1, input_shape=data.shape[1:],activation='relu')))
model.add((Conv2D(32, sizeOfFilter1,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))

model.add((Conv2D(64, sizeOfFilter2,activation='relu')))
model.add((Conv2D(64, sizeOfFilter2,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(noOfNode, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [13]:
checkpoint = ModelCheckpoint('./training/model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
367/367 [==============================] - ETA: 0s - loss: 0.5798 - accuracy: 0.6621

INFO:tensorflow:Assets written to: ./training\model-001.model\assets


INFO:tensorflow:Assets written to: ./training\model-001.model\assets


367/367 [==============================] - 30s 73ms/step - loss: 0.5798 - accuracy: 0.6621 - val_loss: 0.3235 - val_accuracy: 0.8796
Epoch 2/20
366/367 [============================>.] - ETA: 0s - loss: 0.2563 - accuracy: 0.9084

INFO:tensorflow:Assets written to: ./training\model-002.model\assets


...

INFO:tensorflow:Assets written to: ./training\model-008.model\assets


367/367 [==============================] - 27s 74ms/step - loss: 0.1263 - accuracy: 0.9540 - val_loss: 0.1282 - val_accuracy: 0.9567
Epoch 9/20
367/367 [==============================] - 26s 71ms/step - loss: 0.1172 - accuracy: 0.9569 - val_loss: 0.1303 - val_accuracy: 0.9546
Epoch 10/20
367/367 [==============================] - 30s 82ms/step - loss: 0.1145 - accuracy: 0.9598 - val_loss: 0.1289 - val_accuracy: 0.9584
Epoch 11/20
367/367 [==============================] - 31s 82ms/step - loss: 0.1073 - accuracy: 0.9624 - val_loss: 0.1501 - val_accuracy: 0.9437
Epoch 12/20
367/367 [==============================] - 26s 72ms/step - loss: 0.1071 - accuracy: 0.9628 - val_loss: 0.1325 - val_accuracy: 0.9523
Epoch 13/20
366/367 [============================>.] - ETA: 0s - loss: 0.0974 - accuracy: 0.9651

INFO:tensorflow:Assets written to: ./training\model-013.model\assets


INFO:tensorflow:Assets written to: ./training\model-013.model\assets


367/367 [==============================] - 31s 85ms/step - loss: 0.0976 - accuracy: 0.9650 - val_loss: 0.1142 - val_accuracy: 0.9598
Epoch 14/20
367/367 [==============================] - 25s 69ms/step - loss: 0.0949 - accuracy: 0.9654 - val_loss: 0.1143 - val_accuracy: 0.9652
Epoch 15/20
367/367 [==============================] - 25s 69ms/step - loss: 0.0970 - accuracy: 0.9659 - val_loss: 0.1219 - val_accuracy: 0.9584
Epoch 16/20
367/367 [==============================] - 27s 75ms/step - loss: 0.0860 - accuracy: 0.9703 - val_loss: 0.1312 - val_accuracy: 0.9550
Epoch 17/20
367/367 [==============================] - 29s 80ms/step - loss: 0.0831 - accuracy: 0.9716 - val_loss: 0.1218 - val_accuracy: 0.9618
Epoch 18/20
367/367 [==============================] - 31s 84ms/step - loss: 0.0820 - accuracy: 0.9699 - val_loss: 0.1379 - val_accuracy: 0.9502
Epoch 19/20
367/367 [==============================] - 31s 84ms/step - loss: 0.0793 - accuracy: 0.9719 - val_loss: 0.1163 - val_accuracy: 0.96

In [14]:
from keras.models import load_model
import cv2
import numpy as np

In [15]:
model = load_model('./training/model-005.model')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default (1).xml')

cap=cv2.VideoCapture(0)

labels_dict={1:'Male',0:'Female'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [ ]:
while(True):
    
    ret,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,3)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(32,32))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,32,32,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('Result',img)
    key=cv2.waitKey(1)
    
    if (key==27):
        break
    if ret==False:
        break
        
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 21ms/step


...

1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 47ms/step
